# Table of Contents

    1. Introduction

    2. Dataset Preparation
    
    3. Exploratory Data Analysis

    4.  KNN

        4.1 
        4.2 
    

    5. SVD

            5.1. 
            5.2.
            5.3. 
            5.4. 

    

# 1. Introduction

# 2. Dataset Preparation

In [2]:
# Import libraries 
import numpy as np
import pandas as pd 
import semopy 
import seaborn as sns 
import matplotlib.pyplot as plt
import researchpy as rp
import math
import scipy.stats as stats
import statsmodels.stats.api as sms
from statsmodels.stats import power as pwr
from statsmodels.stats.outliers_influence import variance_inflation_factor
import IPython
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import ListedColormap
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans 
from sklearn.datasets import make_blobs
from sklearn import metrics 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
import statsmodels.formula.api as smf
import glob, os
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import GridSearchCV, train_test_split

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import NearestNeighbors

from IPython.display import display
pd.set_option('display.max_columns', None)

In [3]:
#Reading the datasets with post data
df2 = pd.read_csv("jester_items.csv") 
df = pd.read_csv("jester_ratings.csv") 

In [4]:
# Merging datasets according to  "jokeId" 
df = df.merge(right=df2[['jokeId','jokeText']], how='left',on='jokeId')

In [5]:
df.head()

,userId,jokeId,rating,jokeText
0,1,5,0.219,Q.\tWhat's O. J. Simpson's Internet address? \...
1,1,7,-9.281,How many feminists does it take to screw in a ...
2,1,8,-9.281,Q. Did you hear about the dyslexic devil worsh...
3,1,13,-6.781,They asked the Japanese visitor if they have e...
4,1,15,0.875,Q: What did the blind person say when given s...


In [6]:
df.isnull().sum()

userId      0
jokeId      0
rating      0
jokeText    0
dtype: int64

In [7]:
df.dtypes

userId        int64
jokeId        int64
rating      float64
jokeText     object
dtype: object

In [10]:
df.describe()

,userId,jokeId,rating
count,1.761439e+06,1.761439e+06,1.761439e+06
mean,3.272322e+04,7.071133e+01,1.618602e+00
std,1.828011e+04,4.600790e+01,5.302608e+00
min,1.000000e+00,5.000000e+00,-1.000000e+01
25%,1.720200e+04,2.100000e+01,-2.031000e+00
50%,3.480800e+04,6.900000e+01,2.219000e+00
75%,4.730600e+04,1.120000e+02,5.719000e+00
max,6.397800e+04,1.500000e+02,1.000000e+01


# 3. Exploratory Data Analysis

# 4. KNN

In [42]:
# Split the dataset into train and test sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [43]:
joke_user=train_data.pivot(index='userId', columns= 'jokeId',values='rating').fillna(0)
joke_user.head()

jokeId,5,7,8,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.219,-9.281,-9.281,-6.781,0.875,-9.656,-9.031,-7.469,-8.719,-9.156,-7.188,-8.781,-8.531,-7.906,0.000,0.0,8.781,0.000,8.781,0.000,8.781,8.781,0.000,0.000,8.781,0.000,0.000,0.000,0.000,0.00,0.000,0.062,0.0,0.0,0.000,0.000,0.00,0.000,0.000,9.906,0.0,0.062,8.781,8.688,0.000,0.00,0.0,0.0,0.00,0.0,0.031,0.219,0.000,0.0,8.781,0.000,0.0,-0.125,8.688,0.0,0.0,0.000,0.000,0.0,0.0,9.312,0.000,0.000,0.0,0.062,0.125,0.000,8.781,0.000,0.000,0.000,8.000,0.000,9.812,0.00,0.000,3.625,9.312,0.0,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.750,-5.0,2.938,2.00,-0.156,2.031,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.688,0.0,0.0,0.0,0.0,0.0,0.0,3.344,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-9.688,9.938,9.531,9.938,0.406,3.719,9.656,-2.688,-9.562,-9.125,9.844,0.000,0.000,0.000,0.000,0.0,9.781,9.812,9.906,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.0,0.0,0.000,0.000,0.00,9.281,4.969,9.938,0.0,0.000,8.000,-4.312,0.000,0.00,0.0,0.0,0.00,0.0,-0.781,4.750,0.000,0.0,3.688,-2.125,0.0,0.000,-5.000,0.0,0.0,4.781,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.000,-0.312,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.0,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.000,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-9.844,-9.844,-7.219,-2.031,-9.938,-9.969,-9.875,-9.812,-9.781,-6.844,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,-9.812,-9.781,0.000,0.000,-9.812,0.062,0.000,0.00,0.000,0.000,0.0,0.0,0.000,0.000,0.00,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.00,0.0,0.0,0.00,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.0,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.344,0.0,0.000,1.25,0.000,0.000,0.0,-9.812,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-5.812,-4.500,-4.906,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.0,0.0,0.000,0.000,0.00,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.00,0.0,0.0,0.00,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.0,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.000,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6.906,4.750,-5.906,-0.406,-4.031,0.000,6.219,5.656,0.000,5.406,6.375,7.031,0.094,0.000,0.906,0.0,0.438,2.281,1.188,0.281,3.219,0.000,0.000,0.531,3.812,4.469,0.000,2.812,4.812,0.25,4.438,5.562,0.0,0.5,3.969,5.781,6.25,0.344,4.281,1.969,0.0,0.000,2.594,1.188,0.781,2.75,0.0,0.0,-0.25,0.0,1.688,0.625,0.156,0.0,0.000,0.000,0.0,0.781,0.562,0.0,0.0,8.000,3.562,0.0,0.0,7.062,0.875,4.812,0.0,-6.438,3.906,4.156,2.875,-0.031,0.469,0.344,3.188

In [44]:
# Preprocess the data into a sparse matrix format
csrtrain = csr_matrix(joke_user.values)

In [45]:
joke_user.shape

(58782, 140)

In [46]:
csrtrain.shape

(58782, 140)

In [47]:
# Define the parameter grid for grid search
'''
param_grid = {'n_neighbors': [10, 20, 30],
              'algorithm': ['brute'],
              'metric': ['cosine', 'euclidean']}

# Perform grid search with cross-validation
knn = KNeighborsRegressor()
grid_search = GridSearchCV(knn, param_grid, cv=5, n_jobs=-1, scoring='neg_root_mean_squared_error')
grid_search.fit(csrtrain,csrtrain)

'''

"\nparam_grid = {'n_neighbors': [10, 20, 30],\n              'algorithm': ['brute'],\n              'metric': ['cosine', 'euclidean']}\n\n# Perform grid search with cross-validation\nknn = KNeighborsRegressor()\ngrid_search = GridSearchCV(knn, param_grid, cv=5, n_jobs=-1, scoring='neg_root_mean_squared_error')\ngrid_search.fit(csrtrain,csrtrain)\n\n"

In [48]:

# Train a KNN model with the best hyperparameters
knn2 = NearestNeighbors(n_neighbors=10,
                           algorithm='brute',
                           metric='cosine')
knn2.fit( csrtrain)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [49]:
# Recommending Joke to Random User

In [50]:
user_ids = train_data['userId'].unique()
user_ids.sort()
user_ids=user_ids.tolist()
random_user_id= np.random.choice(user_ids)
index = user_ids.index(random_user_id)
print ('Randomly selected user id: ', random_user_id)

Randomly selected user id:  21098


In [51]:
distances, indices = knn2.kneighbors(joke_user.iloc[index,:].values.reshape(1,-1), n_neighbors=10)

In [52]:
for i in range(0,len(distances.flatten())):
    if i==0:
        print('Recommandations for User',str(random_user_id),' \n')
    else:
        print( 'Most Similar User',i,': User_Id ' +  str(joke_user.index[indices.flatten()[i]]),' with distance of ',str(distances.flatten()[i])  )


Recommandations for User 21098  

Most Similar User 1 : User_Id 17294  with distance of  0.20928293010389454
Most Similar User 2 : User_Id 22375  with distance of  0.2576915225873856
Most Similar User 3 : User_Id 28628  with distance of  0.27024577875196243
Most Similar User 4 : User_Id 7583  with distance of  0.2939643868908507
Most Similar User 5 : User_Id 21899  with distance of  0.3005363078923847
Most Similar User 6 : User_Id 39760  with distance of  0.31324231612487907
Most Similar User 7 : User_Id 26233  with distance of  0.314535838596149
Most Similar User 8 : User_Id 27842  with distance of  0.32047264813260257
Most Similar User 9 : User_Id 5240  with distance of  0.3451076500515693


In [53]:
mostsimilaruserjokes = train_data[train_data['userId'] ==joke_user.index[indices.flatten()[1]]]
mostsimilaruserjokes

,userId,jokeId,rating,jokeText
442617,17294,8,3.250,Q. Did you hear about the dyslexic devil worsh...
442622,17294,18,4.062,A dog walks into Western Union and asks the cl...
442633,17294,89,6.719,A radio conversation of a US naval \nship with...
442624,17294,35,4.812,An explorer in the deepest Amazon suddenly fin...
442619,17294,15,2.031,Q: What did the blind person say when given s...
442634,17294,72,7.719,"On the first day of college, the Dean addresse..."
442626,17294,114,7.000,Sherlock Holmes and Dr. Watson go on a camping...
442630,17294,105,7.969,A couple of hunters are out in the woods in th...
442623,17294,19,3.562,Q: If a person who speaks three languages is c...
442628,17294,21,8.125,What's the difference between a used tire and ...


In [54]:
similaruser=[]
jokes_ratedbysimilarusers=[]


for i in range(0,len(distances.flatten())):
    
    similaruser.append(joke_user.index[indices.flatten()[i]])


In [55]:

# filter the data to only include rows for the selected user ID
similaruser_data = train_data[train_data["userId"].isin(similaruser)]
similaruser_data

,userId,jokeId,rating,jokeText
564469,21899,132,3.812,Mickey Mouse is having a nasty divorce with Mi...
674334,28628,16,2.281,Q. What is orange and sounds like a parrot? \...
220722,7583,15,2.219,Q: What did the blind person say when given s...
1063451,39760,117,0.562,A man joins a big corporate empire as a traine...
552610,21098,138,4.562,WASHINGTON (Reuters) - A tragic fire on Monday...
...,...,...,...,...
674335,28628,17,1.250,How many men does it take to screw in a light ...
220725,7583,18,2.250,A dog walks into Western Union and asks the cl...
573570,22375,110,6.344,"One day, a professor was giving a big test to ..."
636925,26233,35,5.281,An explorer in the deepest Amazon suddenly fin...


In [56]:
similaruser_data['avg_rating']  = similaruser_data.groupby('jokeId')['rating'].transform('mean')


/var/folders/y7/_v7_fsx55jngwk83d1p07t8r0000gn/T/ipykernel_841/1090668554.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similaruser_data['avg_rating']  = similaruser_data.groupby('jokeId')['rating'].transform('mean')


In [57]:
similaruser_data

,userId,jokeId,rating,jokeText,avg_rating
564469,21899,132,3.812,Mickey Mouse is having a nasty divorce with Mi...,6.773625
674334,28628,16,2.281,Q. What is orange and sounds like a parrot? \...,1.713333
220722,7583,15,2.219,Q: What did the blind person say when given s...,-1.209714
1063451,39760,117,0.562,A man joins a big corporate empire as a traine...,4.196286
552610,21098,138,4.562,WASHINGTON (Reuters) - A tragic fire on Monday...,5.578000
...,...,...,...,...,...
674335,28628,17,1.250,How many men does it take to screw in a light ...,1.906167
220725,7583,18,2.250,A dog walks into Western Union and asks the cl...,3.826222
573570,22375,110,6.344,"One day, a professor was giving a big test to ...",4.965000
636925,26233,35,5.281,An explorer in the deepest Amazon suddenly fin...,6.046750


In [58]:
# Jokes rated  by similar users and not rated by selected random user

jokesfromsimilarusers= similaruser_data[["jokeId",'jokeText','avg_rating']].drop_duplicates(subset=['jokeId'])

jokesratedbyselecteduser=train_data[(train_data['userId'] == random_user_id)]['jokeText'].tolist()

jokesfromsimilarusers = jokesfromsimilarusers[~jokesfromsimilarusers['jokeText'].isin(jokesratedbyselecteduser)]

jokesfromsimilarusers


,jokeId,jokeText,avg_rating
220722,15,Q: What did the blind person say when given s...,-1.209714
1063451,117,A man joins a big corporate empire as a traine...,4.196286
564462,18,A dog walks into Western Union and asks the cl...,3.826222
169136,145,"A blonde, brunette, and a red head are all lin...",1.625000
674351,106,An engineer dies and reports to the pearly gat...,1.699800
636922,17,How many men does it take to screw in a light ...,1.906167
674359,150,"In an interview with David Letterman, Carter p...",0.656000
657109,50,A guy goes into confession and says to the pri...,3.421500
573577,29,An old Scotsmen is sitting with a younger Scot...,4.625000
169142,48,"The graduate with a Science degree asks, ""Why ...",6.885333


In [59]:
jokeidlist=jokesfromsimilarusers["jokeId"].tolist()
ratelist= jokesfromsimilarusers["avg_rating"].tolist()
titlelist= jokesfromsimilarusers["jokeText"].tolist()

In [60]:
maxrating=0


for i in range(0,len(jokeidlist)):
   if (ratelist[i]>= maxrating):
      
         recommended_joke=titlelist[i]
         maxrating=ratelist[i]
      

print('For User ',random_user_id, 'recommended joke is: ', recommended_joke,' \n', ' with Rating by Similar Users: ' , jokesfromsimilarusers[(jokesfromsimilarusers['jokeText'] == recommended_joke)]['avg_rating'].iloc[0],' \n', ' with General Rating: ' , round(train_data[(train_data['jokeText'] == recommended_joke)].groupby('jokeId')['rating'].transform('mean').iloc[0],1))



For User  21098 recommended joke is:  The graduate with a Science degree asks, "Why does it work?"
The graduate with an Engineering degree asks, "How does it work?"
The graduate with an Accounting degree Asks, "How much will it cost?" 
The graduate with a  Liberal Arts degree asks, "Do you want fries 
with  that?"
  
  with Rating by Similar Users:  6.885333333333333  
  with General Rating:  2.7
